In [16]:
#2016112083 김연웅 HW4

import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset,ConcatDataset
import numpy as np


# Configure Random seeds
torch.manual_seed(777)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(777)

# Define DNN Model
class MyModel(torch.nn.Module):
    
    def __init__(self):
        super(MyModel,self).__init__()
        self.l1=torch.nn.Linear(100,30,bias=True)
        self.l2=torch.nn.Linear(30,15,bias=True)
        self.l3 = torch.nn.Linear(15,1,bias=True)
            
       
        self.Sigmoid = torch.nn.Sigmoid()
        self.ReLU=torch.nn.ReLU()
        
        self.dropout = torch.nn.Dropout(p=0.2)

        
    def forward(self,x):
        x = self.dropout(self.ReLU((self.l1(x))))
        x = self.dropout(self.ReLU((self.l2(x))))
        y_pred = self.Sigmoid(self.l3(x))

        return y_pred

# Define custom dataset - train set
class HW4_trainDataset(Dataset):
    
    def __init__(self):
        train_x =np.loadtxt('./HW4_trainX.csv',delimiter=',',dtype=np.float32)
        train_y =np.loadtxt('./HW4_trainY.csv',delimiter=',',dtype=np.float32).reshape(-1,1)
        
        self.len= train_x.shape[0]
        self.x_data=torch.from_numpy(train_x)
        self.y_data=torch.from_numpy(train_y)
        
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

# Define custom dataset - test set    
class HW4_testDataset(Dataset):
    
    def __init__(self):
        test_x =np.loadtxt('./HW4_testX.csv',delimiter=',',dtype=np.float32)
        test_y =np.loadtxt('./HW4_testY.csv',delimiter=',',dtype=np.float32).reshape(-1,1)
        
        self.len= test_x.shape[0]
        self.x_data=torch.from_numpy(test_x)
        self.y_data=torch.from_numpy(test_y)
        
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

# Create model and print number of parameters
model = MyModel()
params = list(model.parameters())
print("The number of parameters:", sum([p.numel() for p in model.parameters() if p.requires_grad]), "elements")


# Hyper parameter setting
batch_size = 1000
learning_rate =0.01
weight_decay = 0.02
num_epoch = 500
momentum = 0.95
# Create Loss function and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay=weight_decay, momentum = momentum)


# Create datsets and dataloader
trainset = HW4_trainDataset()
testset = HW4_testDataset()
train_loader =DataLoader(trainset, batch_size = batch_size, shuffle = True)
test_loader= DataLoader(testset, batch_size = batch_size, shuffle = False)


The number of parameters: 3511 elements


In [14]:

# train the model
valid_loss_min = int(1e9)
for epoch in range(num_epoch):
    
    train_loss=0.0
    valid_loss=0.0
    
    # Train 
    
    model.train()
    # loop for ever batch set in train dataset loader
    for i, (data, target) in enumerate(train_loader):
        
        # initialize optimizer
        optimizer.zero_grad()
        
        # train
        y_pred = model(data)
        
        # caculate batch loss
        loss = criterion(y_pred, target)
        
        # update
        loss.backward()
        optimizer.step()
    
        # Record train loss
        train_loss += loss.item() 
    
    # Validate 
    
    model.eval()
    # loop for ever batch set in test dataset loader
    for i, (data, target) in enumerate(test_loader):
     
        y_pred = model(data)
        # calculate the batch loss
        loss = criterion(y_pred, target)
        
        # calculate validataion loss
        valid_loss += loss.item()
        
        
    # calculate average losses
    train_loss = train_loss/len(train_loader)
    valid_loss = valid_loss/len(test_loader)
    
    # print training/validation results
    print('Epoch: {}     Training Loss: {}    Validation Loss: {}'.format(epoch,train_loss, valid_loss))
    
    # if trained model perform best vaildation loss, then save it to the checkpoint
    if valid_loss <= valid_loss_min:
        print('Decreased Validation Loss ({} ==> {})   < model saved >'.format(valid_loss_min,valid_loss))
        
        checkpoint = {
            'state_dict': model.state_dict(),
        }
        torch.save(checkpoint, './bestModel.pt')
        valid_loss_min = valid_loss


        

    

Epoch: 0     Training Loss: 0.629828829318285    Validation Loss: 0.606233075261116
Decreased Validation Loss (1000000000 ==> 0.606233075261116)   < model saved >
Epoch: 1     Training Loss: 0.6044873967766762    Validation Loss: 0.6002116352319717
Decreased Validation Loss (0.606233075261116 ==> 0.6002116352319717)   < model saved >
Epoch: 2     Training Loss: 0.5979300104081631    Validation Loss: 0.5952985286712646
Decreased Validation Loss (0.6002116352319717 ==> 0.5952985286712646)   < model saved >
Epoch: 3     Training Loss: 0.5943762473762035    Validation Loss: 0.5913120359182358
Decreased Validation Loss (0.5952985286712646 ==> 0.5913120359182358)   < model saved >
Epoch: 4     Training Loss: 0.5894307643175125    Validation Loss: 0.5892904996871948
Decreased Validation Loss (0.5913120359182358 ==> 0.5892904996871948)   < model saved >
Epoch: 5     Training Loss: 0.5858632437884808    Validation Loss: 0.584944948554039
Decreased Validation Loss (0.5892904996871948 ==> 0.58494

Epoch: 52     Training Loss: 0.43869162537157536    Validation Loss: 0.4503941610455513
Epoch: 53     Training Loss: 0.4385774936527014    Validation Loss: 0.44969670474529266
Decreased Validation Loss (0.44980181008577347 ==> 0.44969670474529266)   < model saved >
Epoch: 54     Training Loss: 0.4369567036628723    Validation Loss: 0.4482212960720062
Decreased Validation Loss (0.44969670474529266 ==> 0.4482212960720062)   < model saved >
Epoch: 55     Training Loss: 0.433780325576663    Validation Loss: 0.4476252421736717
Decreased Validation Loss (0.4482212960720062 ==> 0.4476252421736717)   < model saved >
Epoch: 56     Training Loss: 0.4334953036159277    Validation Loss: 0.4484350234270096
Epoch: 57     Training Loss: 0.4357123300433159    Validation Loss: 0.4465237110853195
Decreased Validation Loss (0.4476252421736717 ==> 0.4465237110853195)   < model saved >
Epoch: 58     Training Loss: 0.4357531238347292    Validation Loss: 0.44831616431474686
Epoch: 59     Training Loss: 0.435

Epoch: 137     Training Loss: 0.431545939296484    Validation Loss: 0.44679366052150726
Epoch: 138     Training Loss: 0.4301741402596235    Validation Loss: 0.4459208846092224
Epoch: 139     Training Loss: 0.42860776744782925    Validation Loss: 0.44263654202222824
Decreased Validation Loss (0.44327565282583237 ==> 0.44263654202222824)   < model saved >
Epoch: 140     Training Loss: 0.4244915582239628    Validation Loss: 0.4451066330075264
Epoch: 141     Training Loss: 0.42985717579722404    Validation Loss: 0.44460250437259674
Epoch: 142     Training Loss: 0.42674767412245274    Validation Loss: 0.443964920938015
Epoch: 143     Training Loss: 0.4269796907901764    Validation Loss: 0.4451286643743515
Epoch: 144     Training Loss: 0.42848474346101284    Validation Loss: 0.44493769109249115
Epoch: 145     Training Loss: 0.4262015949934721    Validation Loss: 0.4446178376674652
Epoch: 146     Training Loss: 0.4319097213447094    Validation Loss: 0.4458685517311096
Epoch: 147     Training 

Epoch: 228     Training Loss: 0.4291014540940523    Validation Loss: 0.44699088484048843
Epoch: 229     Training Loss: 0.4274590443819761    Validation Loss: 0.4444108158349991
Epoch: 230     Training Loss: 0.43112028762698174    Validation Loss: 0.4484073668718338
Epoch: 231     Training Loss: 0.43003251776099205    Validation Loss: 0.4462350681424141
Epoch: 232     Training Loss: 0.430971622467041    Validation Loss: 0.44624190032482147
Epoch: 233     Training Loss: 0.4284768234938383    Validation Loss: 0.44766443222761154
Epoch: 234     Training Loss: 0.4297017380595207    Validation Loss: 0.4466732293367386
Epoch: 235     Training Loss: 0.43108316510915756    Validation Loss: 0.44750188291072845
Epoch: 236     Training Loss: 0.4271904230117798    Validation Loss: 0.4457057788968086
Epoch: 237     Training Loss: 0.4279842171818018    Validation Loss: 0.44559265673160553
Epoch: 238     Training Loss: 0.4284876845777035    Validation Loss: 0.4444227069616318
Epoch: 239     Training L

Epoch: 321     Training Loss: 0.42618444934487343    Validation Loss: 0.44384197145700455
Epoch: 322     Training Loss: 0.4246606696397066    Validation Loss: 0.44440554082393646
Epoch: 323     Training Loss: 0.4258111771196127    Validation Loss: 0.44653429090976715
Epoch: 324     Training Loss: 0.4250571969896555    Validation Loss: 0.44648534059524536
Epoch: 325     Training Loss: 0.42730789817869663    Validation Loss: 0.44427918642759323
Epoch: 326     Training Loss: 0.42249152436852455    Validation Loss: 0.4457308277487755
Epoch: 327     Training Loss: 0.4254183601588011    Validation Loss: 0.44787176698446274
Epoch: 328     Training Loss: 0.4265903178602457    Validation Loss: 0.4462573602795601
Epoch: 329     Training Loss: 0.4310387447476387    Validation Loss: 0.4490159600973129
Epoch: 330     Training Loss: 0.4239142816513777    Validation Loss: 0.44601839780807495
Epoch: 331     Training Loss: 0.42906054854393005    Validation Loss: 0.4481397792696953
Epoch: 332     Traini

Epoch: 414     Training Loss: 0.4253734238445759    Validation Loss: 0.44745396822690964
Epoch: 415     Training Loss: 0.4286520015448332    Validation Loss: 0.44790808111429214
Epoch: 416     Training Loss: 0.4265539962798357    Validation Loss: 0.446487121284008
Epoch: 417     Training Loss: 0.4295752849429846    Validation Loss: 0.4451846480369568
Epoch: 418     Training Loss: 0.4237330723553896    Validation Loss: 0.4477066323161125
Epoch: 419     Training Loss: 0.4253308419138193    Validation Loss: 0.4462487921118736
Epoch: 420     Training Loss: 0.4251353908330202    Validation Loss: 0.44654639810323715
Epoch: 421     Training Loss: 0.42693736776709557    Validation Loss: 0.44693107157945633
Epoch: 422     Training Loss: 0.42926207557320595    Validation Loss: 0.44848664104938507
Epoch: 423     Training Loss: 0.4257946517318487    Validation Loss: 0.4471297487616539
Epoch: 424     Training Loss: 0.42710419557988644    Validation Loss: 0.44690562039613724
Epoch: 425     Training 

In [15]:
# Load best performance model
checkpoint = torch.load('./bestModel.pt')
trained_model = MyModel()



trained_model.load_state_dict(checkpoint['state_dict'])

# evaluate
trained_model.eval()
with torch.no_grad():
    
    # load test dataset
    test_x = torch.from_numpy(np.loadtxt('./HW4_testX.csv',delimiter=',',dtype=np.float32))
    test_y = torch.from_numpy(np.loadtxt('./HW4_testY.csv',delimiter=',',dtype=np.float32).reshape(-1,1))
    
    # evaluate using weighted f1 score.
    y_pred = trained_model(test_x)
    yp = y_pred.detach().numpy()
    yp = [1.0 if x > 0.5 else 0.0 for x in yp]

    print("weighted F1:", f1_score(test_y, yp, average='weighted'))

weighted F1: 0.8329938841330211


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-7.228919,6.463900,0.472254,8.241039,-7.986439,4.419828,7.937655,-4.752053,-2.932872,-3.783504,...,1.103139,6.542378,5.916002,0.869227,-0.654664,-3.400430,3.665243,-5.362047,0.960563,2.730822
1,0.680726,-4.168741,7.034253,-5.463813,-0.575878,-4.839989,9.169139,1.040742,-7.860359,-0.310231,...,6.956896,-7.645589,-5.353952,-1.718410,2.977554,1.760390,6.545843,-5.365576,6.347999,-10.805965
2,1.887534,2.845743,-2.861815,-7.900907,2.281003,-3.394852,-1.058811,-4.785132,-1.281071,2.939634,...,4.970719,1.772561,-4.717122,0.332202,9.332535,-2.939443,-1.905870,-3.286218,3.845007,0.604373
3,-10.019826,-3.790430,-4.893445,-2.197804,-3.053076,1.294890,-5.630816,-7.947301,4.348945,10.246729,...,-2.535092,0.822923,-2.708332,-6.332679,10.530745,-2.172284,-7.045495,8.866513,-2.950878,0.826856
4,-1.814451,-9.006926,-1.514610,-2.765532,6.146830,2.998480,9.140896,8.670193,8.397481,5.403070,...,0.420641,9.998662,-2.859708,3.442361,12.766651,3.086012,-5.230854,-4.750442,1.549153,-0.635022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,3.974874,-2.993205,-2.683005,1.079106,0.613689,0.035055,-4.698192,-3.738479,0.801668,-6.480069,...,-0.499226,0.758832,-1.238975,4.747481,0.468105,4.014269,-2.281803,12.485242,3.654442,4.299384
15996,1.519937,-9.648294,-0.846710,5.303586,6.897678,-4.581512,-10.776951,3.953178,0.909904,-6.828432,...,1.933507,-2.223882,-5.727911,10.852921,2.892385,-0.214741,10.700461,-7.454326,-2.699429,4.263726
15997,0.866765,-0.082205,2.748877,-0.690968,4.544461,-2.363283,3.472220,2.743873,-4.181521,-8.709196,...,-14.731414,-0.359130,9.178993,-15.711183,-3.151777,4.001902,-7.767521,4.687436,4.230262,1.366639
15998,1.117300,2.844754,-2.626271,-2.040471,-0.352758,8.903402,4.876278,-2.957271,-5.091562,0.800120,...,3.066929,3.055679,6.179786,-0.095293,7.500524,0.374477,6.998742,6.433831,-14.591685,1.387973
